In [1]:
import os

In [2]:
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\winequality_prediction\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\winequality_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str

In [6]:
from src.WineQualityPrediction.constants import *
from src.WineQualityPrediction.utils.common import read_yaml, create_directories
from src.WineQualityPrediction.utils.my_logging import logger
from src.WineQualityPrediction.utils.my_exception import CustomException

In [7]:
class ConfigurationManager:
    """
    Manages the configuration for the project by reading YAML files
    and providing structured configurations.

    Attributes:
        config (ConfigBox): Parsed configuration file.
        params (ConfigBox): Parsed parameters file.
        schema (ConfigBox): Parsed schema file.
    """

    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH,
                 schema_filepath: Path = SCHEMA_FILE_PATH):
        
        """
        Initializes the ConfigurationManager by loading YAML files and creating directories.

        Args:
            config_filepath (Path): Path to the main configuration file.
            params_filepath (Path): Path to the parameters configuration file.
            schema_filepath (Path): Path to the schema configuration file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        Provides the configuration for the model trainer.

        Returns:
            ModelTrainerConfig: Configuration for the model trainer.
        """
        config = self.config.model_trainer
        param = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name
        )
        return model_trainer_config

In [8]:
log_path = 'log\log_file.log'

In [9]:
import pandas as pd
import os
import sys
import logging
from src.WineQualityPrediction.utils.my_logging import logger
from src.WineQualityPrediction.utils.my_exception import CustomException
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        """
        Initializes the ModelTrainer class with a given configuration.

        Parameters:
            config (ModelTrainerConfig): An object containing the configuration 
                                        parameters for training, including paths 
                                        to datasets, model hyperparameters, and 
                                        save paths.
        """
        self.config = config

    def train(self):
        logger(log_path,logging.INFO, "Training the model Started...")

        """
        Trains the ElasticNet regression model using the provided configuration.

        Steps:
            1. Reads training and testing datasets from the specified paths.
            2. Separates features (X) and target (Y) from the datasets.
            3. Initializes and trains the ElasticNet model with specified hyperparameters.
            4. Saves the trained model to the configured directory.

        Dependencies:
            - pandas (pd): Used to read the CSV datasets.
            - sklearn.linear_model.ElasticNet: The regression model used for training.
            - joblib: Used for saving the trained model to disk.
            - os: Handles file paths for saving the model.

        Raises:
            FileNotFoundError: If the training or testing data files are not found.
            ValueError: If the target column is not present in the datasets.

        Notes:
            Ensure that the `train_data_path` and `test_data_path` in the configuration point to valid CSV files.
            The directory specified by `root_dir` should exist or be created beforehand.
        """     
        logger(log_path,logging.INFO, "Reading the datasets...")   
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        logger(log_path,logging.INFO, "Separating features and target...")
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        logger(log_path,logging.INFO, "strart the model building... ")
        logger(log_path,logging.INFO, "Training the model...")
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        logger(log_path,logging.INFO, "Model training completed...")
        logger(log_path,logging.INFO, "Saving the model...")
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        logger(log_path,logging.INFO, "Model saved...")


In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise CustomException(e,sys)

[2025-01-15 14:48:24] - INFO - Yaml file read successfully
[2025-01-15 14:48:24] - INFO - Yaml file read successfully
[2025-01-15 14:48:24] - INFO - Yaml file read successfully
[2025-01-15 14:48:24] - INFO - Directory created at artifacts
[2025-01-15 14:48:24] - INFO - Directory created at artifacts/model_trainer
[2025-01-15 14:48:24] - INFO - Training the model Started...
[2025-01-15 14:48:24] - INFO - Reading the datasets...
[2025-01-15 14:48:25] - INFO - Separating features and target...
[2025-01-15 14:48:25] - INFO - strart the model building... 
[2025-01-15 14:48:25] - INFO - Training the model...
[2025-01-15 14:48:25] - INFO - Model training completed...
[2025-01-15 14:48:25] - INFO - Saving the model...
[2025-01-15 14:48:25] - INFO - Model saved...
